<a href="https://colab.research.google.com/github/tetiana-kholod/a-b-test/blob/main/a_b_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###In this work, the statistical significance of differences in key metrics between test and control groups was evaluated across four separate A/B tests, and the results were summarized in a structured final table.

###[View Tableau Dashboard](https://public.tableau.com/app/profile/tetiana.kholod/viz/ABtestingtoolCalculator/ABtest)


In [242]:
#!pip install --upgrade google-cloud-bigquery


In [243]:
from google.cloud import bigquery
from google.oauth2 import service_account
from google.colab import auth

import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np
import pandas as pd
from scipy.stats import normaltest, shapiro
from scipy.stats import norm

In [244]:
auth.authenticate_user()

In [245]:
client = bigquery.Client(project="data-analytics-mate")

In [246]:
query = """

with session_info as (
  SELECT
    s.date,
    s.ga_session_id,
    sp,country,
    sp.device,
    sp.continent,
    sp.channel,
    ab.test,
    ab.test_group
  FROM `DA.ab_test` as ab
  join `DA.session_params` as sp
  on ab.ga_session_id = sp.ga_session_id
  join `DA.session` as s
  on s.ga_session_id = ab.ga_session_id
),
session_with_order as (
  SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   count (DISTINCT o.ga_session_id) as session_with_order
  FROM `DA.order` as o
  join session_info
  ON o.ga_session_id = session_info.ga_session_id
  GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group
),
events as(
  SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   sp.event_name,
   count (sp.ga_session_id) as events_cnt
  FROM `DA.event_params` as sp
  JOIN session_info
  ON session_info.ga_session_id = sp.ga_session_id
  GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   sp.event_name
),
sessions as (
  SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   count (DISTINCT session_info.ga_session_id) as sessions_cnt
  FROM session_info
  GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group
),
accounts as(
  SELECT
   session_info.date,
   session_info.country,
   session_info.continent,
   session_info.device,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   count (acs.ga_session_id) as new_accounts_cnt
  FROM `DA.account_session` as acs
  JOIN session_info
  ON session_info.ga_session_id = acs.ga_session_id
  GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group
)


SELECT
session_with_order.date,
   session_with_order.continent,
   session_with_order.country,
   session_with_order.device,
   session_with_order.channel,
   session_with_order.test,
   session_with_order.test_group,
   'session_with_order' as event_name,
   session_with_order.session_with_order as value
FROM session_with_order
UNION ALL


SELECT
events.date,
   events.continent,
   events.country,
   events.device,
   events.channel,
   events.test,
   events.test_group,
   events.event_name,
   events.events_cnt as value
FROM events


UNION ALL
SELECT
   accounts.date,
   accounts.continent,
   accounts.country,
   accounts.device,
   accounts.channel,
   accounts.test,
   accounts.test_group,
   'new_accounts' as event_name,
   accounts.new_accounts_cnt as value
FROM accounts


UNION ALL
SELECT
   sessions.date,
   sessions.continent,
   sessions.country,
   sessions.device,
   sessions.channel,
   sessions.test,
   sessions.test_group,
   'sessions' as event,
   sessions_cnt as value
   FROM sessions
"""

In [247]:
query_job = client.query(query)  # Execution of SQL query
results = query_job.result()  # Waiting for query to complete

In [248]:
data = results.to_dataframe()

In [249]:
data.head(1)


,date,continent,country,device,channel,test,test_group,event_name,value
0,2020-12-08,Asia,Palestine,desktop,Direct,4,2,new_accounts,1


#METRICS

add_payment_info / session  
add_shipping_info / session  
begin_checkout / session  
new_accounts / session

In [250]:
#data["event_name"].unique()

In [251]:
#List of Events
events_list = list(data["event_name"].unique())
print(events_list)

['new_accounts', 'session_with_order', 'user_engagement', 'session_start', 'first_visit', 'page_view', 'scroll', 'view_item', 'view_promotion', 'view_search_results', 'begin_checkout', 'add_shipping_info', 'add_to_cart', 'select_item', 'select_promotion', 'add_payment_info', 'click', 'sessions', 'view_item_list']


In [252]:
#to add columns for events (1 / 0)
results = data
events_list = list(data["event_name"].unique())
for event in events_list:
    results[event] = ((results['event_name'] == event).astype(int)) * results['value']
results.head()

,date,continent,country,device,channel,test,test_group,event_name,value,new_accounts,...,view_search_results,begin_checkout,add_shipping_info,add_to_cart,select_item,select_promotion,add_payment_info,click,sessions,view_item_list
0,2020-12-08,Asia,Palestine,desktop,Direct,4,2,new_accounts,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2020-12-08,Asia,Palestine,desktop,Direct,3,2,new_accounts,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2020-11-06,Americas,Puerto Rico,desktop,Social Search,2,2,new_accounts,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2020-11-06,Americas,Puerto Rico,desktop,Social Search,1,1,new_accounts,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2020-12-08,Europe,Croatia,desktop,Direct,4,2,new_accounts,1,1,...,0,0,0,0,0,0,0,0,0,0


In [253]:
#Define the total number of events by test and group
results_grouped_by_test_group = results.groupby(['test', 'test_group'])[events_list].sum()
results_grouped_by_test_group.head()

new_accounts  session_with_order  user_engagement  \
test test_group                                                      
1    1                   3823                4514           171788   
     2                   3681                4526           179081   
2    1                   4165                5102           198266   
     2                   4184                5003           189931   
3    1                   5856                6951           249921   

                 session_start  first_visit  page_view  scroll  view_item  \
test test_group                                                             
1    1                   45905        30596     191543   73244      62335   
     2                   45649        30512     198050   73376      65337   
2    1                   51219        34511     220275   80713      72717   
     2                   50808        34171     212320   81370      68700   
3    1                   71312        50438     286351  110360      93931   

                 view_promotion  view_search_results  begin_checkout  \
test test_group                                                        
1    1                    29188                 3678            3784   
     2                    29117                 3882            4021   
2    1                    32367                 4282            4262   
     2                    31680                 4198            4313   
3    1                    41169                 5764            9532   

                 add_shipping_info  add_to_cart  select_item  \
test test_group                                                
1    1                        3034         1395          543   
     2                        3221         1366          530   
2    1                        3480         2811          905   
     2                        3510         3061          946   
3    1                        5298        17674         8735   

                 select_promotion  add_payment_info  click  sessions  \
test test_group                                                        
1    1                       1275              1988    368     45362   
     2                       1323              2229    353     45193   
2    1                       1477              2344    337     50637   
     2                       1406              2409    413     50244   
3    1                       2020              3623    280     70047   

                 view_item_list  
test test_group                  
1    1                       27  
     2                       24  
2    1                       24  
     2                       29  
3    1                        9

In [254]:
results_grouped_by_test_group = results_grouped_by_test_group.reset_index()
results_grouped_by_test_group['test'] = results_grouped_by_test_group['test'].ffill()
results_grouped_by_test_group.head()

,test,test_group,new_accounts,session_with_order,user_engagement,session_start,first_visit,page_view,scroll,view_item,...,view_search_results,begin_checkout,add_shipping_info,add_to_cart,select_item,select_promotion,add_payment_info,click,sessions,view_item_list
0,1,1,3823,4514,171788,45905,30596,191543,73244,62335,...,3678,3784,3034,1395,543,1275,1988,368,45362,27
1,1,2,3681,4526,179081,45649,30512,198050,73376,65337,...,3882,4021,3221,1366,530,1323,2229,353,45193,24
2,2,1,4165,5102,198266,51219,34511,220275,80713,72717,...,4282,4262,3480,2811,905,1477,2344,337,50637,24
3,2,2,4184,5003,189931,50808,34171,212320,81370,68700,...,4198,4313,3510,3061,946,1406,2409,413,50244,29
4,3,1,5856,6951,249921,71312,50438,286351,110360,93931,...,5764,9532,5298,17674,8735,2020,3623,280,70047,9


METRICS  

add_payment_info / session  

add_shipping_info / session  

begin_checkout / session  

new_accounts / session

In [255]:
test_list = results_grouped_by_test_group["test"].unique()
test_group_list = results_grouped_by_test_group["test_group"].unique()
#events_list_project2 = ["add_payment_info", "add_shipping_info", "begin_checkout", "new_accounts"]

final_table = pd.DataFrame()
for test in test_list:

    results_test_group1 = results_grouped_by_test_group[(results_grouped_by_test_group["test"] == test) & (results_grouped_by_test_group["test_group"] == 1)]
    results_test_group2 = results_grouped_by_test_group[(results_grouped_by_test_group["test"] == test) & (results_grouped_by_test_group["test_group"] == 2)]

    for event in events_list: #events_list_project2


        # to create one row
        new_row_1 = {
        "test_number": test,
            #"test_group": test_group,

        "metric": f"{event} / session",
        "numerator_event": event,
        "denominator_event": "session",
        "numerator_control": results_test_group1[event].iloc[0],
        "denominator_control": results_test_group1["sessions"].iloc[0],
        "conversion_control": results_test_group1[event].iloc[0] / results_test_group1["sessions"].iloc[0],
        "numerator_test": results_test_group2[event].iloc[0],
        "denominator_test": results_test_group2["sessions"].iloc[0],
        "conversion_test": results_test_group2[event].iloc[0] / results_test_group2["sessions"].iloc[0],

        "metric_change_%": (),
        "z_stat": (),
        "p_value": (),
        "significant": (),

        "dimension": "total",
        "dimension_value": "total"
        }

        final_table = pd.concat([final_table, pd.DataFrame([new_row_1])], ignore_index=True)
#final_table["dimension"] = "total"
#final_table["dimension_value"] = "total"

final_table.head()




,test_number,metric,numerator_event,denominator_event,numerator_control,denominator_control,conversion_control,numerator_test,denominator_test,conversion_test,metric_change_%,z_stat,p_value,significant,dimension,dimension_value
0,1,new_accounts / session,new_accounts,session,3823,45362,0.084278,3681,45193,0.081451,(),(),(),(),total,total
1,1,session_with_order / session,session_with_order,session,4514,45362,0.099511,4526,45193,0.100148,(),(),(),(),total,total
2,1,user_engagement / session,user_engagement,session,171788,45362,3.787046,179081,45193,3.962583,(),(),(),(),total,total
3,1,session_start / session,session_start,session,45905,45362,1.011970,45649,45193,1.010090,(),(),(),(),total,total
4,1,first_visit / session,first_visit,session,30596,45362,0.674485,30512,45193,0.675149,(),(),(),(),total,total


In [256]:
devices_list = results["device"].unique()
#print(devices_list)
for device in devices_list:
    results_grouped_by_test_group_devices = results[results["device"] == device].groupby(['test', 'test_group'])[events_list].sum()

    results_grouped_by_test_group_devices = results_grouped_by_test_group_devices.reset_index()
    results_grouped_by_test_group_devices['test'] = results_grouped_by_test_group_devices['test'].ffill()

    for test in test_list:

        results_test_devices_group_1 = results_grouped_by_test_group_devices[(results_grouped_by_test_group_devices["test"] == test) & (results_grouped_by_test_group_devices["test_group"] == 1)]
        results_test_devices_group_2 = results_grouped_by_test_group_devices[(results_grouped_by_test_group_devices["test"] == test) & (results_grouped_by_test_group_devices["test_group"] == 2)]

        for event in events_list:  #events_list_project2


            # to create one row
            new_row_2 = {
            "test_number": test,
            #"test_group": test_group,

            "metric": f"{event} / session",
            "numerator_event": event,
            "denominator_event": "session",
            "numerator_control": results_test_devices_group_1[event].iloc[0],
            "denominator_control": results_test_devices_group_1["sessions"].iloc[0],
            "conversion_control": results_test_devices_group_1[event].iloc[0] / results_test_devices_group_1["sessions"].iloc[0],
            "numerator_test": results_test_devices_group_2[event].iloc[0],
            "denominator_test": results_test_devices_group_2["sessions"].iloc[0],
            "conversion_test": results_test_devices_group_2[event].iloc[0] / results_test_devices_group_2["sessions"].iloc[0],

            "metric_change_%": (),
            "z_stat": (),
            "p_value": (),
            "significant": (),

            "dimension": "device",
            "dimension_value": device
            }

            final_table = pd.concat([final_table, pd.DataFrame([new_row_2])], ignore_index=True)



#final_table["metric_change"] = final_table["conversion_test"] / final_table["conversion_control"] - 1
final_table.head()






,test_number,metric,numerator_event,denominator_event,numerator_control,denominator_control,conversion_control,numerator_test,denominator_test,conversion_test,metric_change_%,z_stat,p_value,significant,dimension,dimension_value
0,1,new_accounts / session,new_accounts,session,3823,45362,0.084278,3681,45193,0.081451,(),(),(),(),total,total
1,1,session_with_order / session,session_with_order,session,4514,45362,0.099511,4526,45193,0.100148,(),(),(),(),total,total
2,1,user_engagement / session,user_engagement,session,171788,45362,3.787046,179081,45193,3.962583,(),(),(),(),total,total
3,1,session_start / session,session_start,session,45905,45362,1.011970,45649,45193,1.010090,(),(),(),(),total,total
4,1,first_visit / session,first_visit,session,30596,45362,0.674485,30512,45193,0.675149,(),(),(),(),total,total


In [257]:
channel_list = results["channel"].unique()

for channel in channel_list:
    results_grouped_by_test_group_channels = results[results["channel"] == channel].groupby(['test', 'test_group'])[events_list].sum()

    results_grouped_by_test_group_channels = results_grouped_by_test_group_channels.reset_index()
    results_grouped_by_test_group_channels['test'] = results_grouped_by_test_group_channels['test'].ffill()

    for test in test_list:

        results_test_channels_group_1 = results_grouped_by_test_group_channels[(results_grouped_by_test_group_channels["test"] == test) & (results_grouped_by_test_group_channels["test_group"] == 1)]
        results_test_channels_group_2 = results_grouped_by_test_group_channels[(results_grouped_by_test_group_channels["test"] == test) & (results_grouped_by_test_group_channels["test_group"] == 2)]

        for event in events_list:  #events_list_project2


            # to create one row
            new_row_3 = {
            "test_number": test,
            #"test_group": test_group,

            "metric": f"{event} / session",
            "numerator_event": event,
            "denominator_event": "session",
            "numerator_control": results_test_channels_group_1[event].iloc[0],
            "denominator_control": results_test_channels_group_1["sessions"].iloc[0],
            "conversion_control": results_test_channels_group_1[event].iloc[0] / results_test_channels_group_1["sessions"].iloc[0],
            "numerator_test": results_test_channels_group_2[event].iloc[0],
            "denominator_test": results_test_channels_group_2["sessions"].iloc[0],
            "conversion_test": results_test_channels_group_2[event].iloc[0] / results_test_channels_group_2["sessions"].iloc[0],

            "metric_change_%": (),
            "z_stat": (),
            "p_value": (),
            "significant": (),

            "dimension": "channel",
            "dimension_value": channel
            }

            final_table = pd.concat([final_table, pd.DataFrame([new_row_3])], ignore_index=True)

#final_table["metric_change"] = final_table["conversion_test"] / final_table["conversion_control"]*100 - 100
final_table.head()

,test_number,metric,numerator_event,denominator_event,numerator_control,denominator_control,conversion_control,numerator_test,denominator_test,conversion_test,metric_change_%,z_stat,p_value,significant,dimension,dimension_value
0,1,new_accounts / session,new_accounts,session,3823,45362,0.084278,3681,45193,0.081451,(),(),(),(),total,total
1,1,session_with_order / session,session_with_order,session,4514,45362,0.099511,4526,45193,0.100148,(),(),(),(),total,total
2,1,user_engagement / session,user_engagement,session,171788,45362,3.787046,179081,45193,3.962583,(),(),(),(),total,total
3,1,session_start / session,session_start,session,45905,45362,1.011970,45649,45193,1.010090,(),(),(),(),total,total
4,1,first_visit / session,first_visit,session,30596,45362,0.674485,30512,45193,0.675149,(),(),(),(),total,total


In [258]:
country_list = results["country"].unique()
#print(devices_list)
for country in country_list:
    results_grouped_by_test_group_countries = results[results["country"] == channel].groupby(['test', 'test_group'])[events_list].sum()

    results_grouped_by_test_group_countries = results_grouped_by_test_group_countries.reset_index()
    results_grouped_by_test_group_countries['test'] = results_grouped_by_test_group_countries['test'].ffill()

    for test in test_list:

        results_test_countries_group_1 = results_grouped_by_test_group_countries[(results_grouped_by_test_group_countries["test"] == test) & (results_grouped_by_test_group_countries["test_group"] == 1)]
        results_test_countries_group_2 = results_grouped_by_test_group_countries[(results_grouped_by_test_group_countries["test"] == test) & (results_grouped_by_test_group_countries["test_group"] == 2)]

        for event in events_list: #events_list_project2


            # to create one row
            new_row_5 = {
            "test_number": test,
            #"test_group": test_group,

            "metric": f"{event} / session",
            "numerator_event": event,
            "denominator_event": "session",
            "numerator_control": results_test_channels_group_1[event].iloc[0],
            "denominator_control": results_test_channels_group_1["sessions"].iloc[0],
            "conversion_control": results_test_channels_group_1[event].iloc[0] / results_test_channels_group_1["sessions"].iloc[0],
            "numerator_test": results_test_channels_group_2[event].iloc[0],
            "denominator_test": results_test_channels_group_2["sessions"].iloc[0],
            "conversion_test": results_test_channels_group_2[event].iloc[0] / results_test_channels_group_2["sessions"].iloc[0],

            "metric_change_%": (),
            "z_stat": (),
            "p_value": (),
            "significant": (),

            "dimension": "country",
            "dimension_value": country
            }

            final_table = pd.concat([final_table, pd.DataFrame([new_row_5])], ignore_index=True)

final_table["metric_change_%"] = final_table["conversion_test"] / final_table["conversion_control"]*100 - 100
final_table.head()

,test_number,metric,numerator_event,denominator_event,numerator_control,denominator_control,conversion_control,numerator_test,denominator_test,conversion_test,metric_change_%,z_stat,p_value,significant,dimension,dimension_value
0,1,new_accounts / session,new_accounts,session,3823,45362,0.084278,3681,45193,0.081451,-3.354299,(),(),(),total,total
1,1,session_with_order / session,session_with_order,session,4514,45362,0.099511,4526,45193,0.100148,0.640785,(),(),(),total,total
2,1,user_engagement / session,user_engagement,session,171788,45362,3.787046,179081,45193,3.962583,4.635176,(),(),(),total,total
3,1,session_start / session,session_start,session,45905,45362,1.011970,45649,45193,1.010090,-0.185807,(),(),(),total,total
4,1,first_visit / session,first_visit,session,30596,45362,0.674485,30512,45193,0.675149,0.098379,(),(),(),total,total


In [259]:

# conversion rate
p1 = final_table["numerator_control"] / final_table["denominator_control"]
p2 = final_table["numerator_test"] / final_table["denominator_test"]

# pooled proportion
p_pool = (final_table["numerator_control"] + final_table["numerator_test"]) / (
    final_table["denominator_control"] + final_table["denominator_test"]
)

# standard error
se = np.sqrt(p_pool * (1 - p_pool) * (1 / final_table["denominator_control"] + 1 / final_table["denominator_test"]))

# z-stat
final_table["z_stat"] = (p2 - p1) / se

# p-value
final_table["p_value"] = 2 * (1 - norm.cdf(np.abs(final_table["z_stat"])))

# signidicance
final_table["significant"] = final_table["p_value"] < 0.05

final_table.head()

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


,test_number,metric,numerator_event,denominator_event,numerator_control,denominator_control,conversion_control,numerator_test,denominator_test,conversion_test,metric_change_%,z_stat,p_value,significant,dimension,dimension_value
0,1,new_accounts / session,new_accounts,session,3823,45362,0.084278,3681,45193,0.081451,-3.354299,-1.542883,0.122859,False,total,total
1,1,session_with_order / session,session_with_order,session,4514,45362,0.099511,4526,45193,0.100148,0.640785,0.320049,0.748931,False,total,total
2,1,user_engagement / session,user_engagement,session,171788,45362,3.787046,179081,45193,3.962583,4.635176,NaN,NaN,False,total,total
3,1,session_start / session,session_start,session,45905,45362,1.011970,45649,45193,1.010090,-0.185807,NaN,NaN,False,total,total
4,1,first_visit / session,first_visit,session,30596,45362,0.674485,30512,45193,0.675149,0.098379,0.213130,0.831225,False,total,total


In [260]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8892 entries, 0 to 8891
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   test_number          8892 non-null   int64  
 1   metric               8892 non-null   object 
 2   numerator_event      8892 non-null   object 
 3   denominator_event    8892 non-null   object 
 4   numerator_control    8892 non-null   int64  
 5   denominator_control  8892 non-null   int64  
 6   conversion_control   8892 non-null   float64
 7   numerator_test       8892 non-null   int64  
 8   denominator_test     8892 non-null   int64  
 9   conversion_test      8892 non-null   float64
 10  metric_change_%      8454 non-null   float64
 11  z_stat               6093 non-null   float64
 12  p_value              6093 non-null   float64
 13  significant          8892 non-null   bool   
 14  dimension            8892 non-null   object 
 15  dimension_value      8892 non-null   o

In [261]:
print(final_table["dimension"].unique())

['total' 'device' 'channel' 'country']


In [262]:
final_table.to_csv('temp100.csv', index=False)
!ls /content


drive  sample_data  temp100.csv


In [263]:
from google.colab import drive
drive.mount('/content/drive')
final_table.to_csv('/content/drive/MyDrive/temp100.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
